In [ ]:
"""
Purpose: To run spine detection on a generic mesh
"""

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from neurd.vdi_microns import volume_data_interface as vdi

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
G = vdi.fetch_segment_obj(12345)
G[1][0].n_synapses_pre
G[1][0].width_array

# Load the data

In [3]:
import trimesh
import numpy as np
from datasci_tools import numpy_utils as nu
from mesh_tools import trimesh_utils as tu

In [4]:
from pathlib import Path
list(Path("./").iterdir())

[PosixPath('.ipynb_checkpoints'),
 PosixPath('Archive'),
 PosixPath('cgal_temp'),
 PosixPath('full_spine_detection.html'),
 PosixPath('full_tri.txt'),
 PosixPath('full_vert.txt'),
 PosixPath('Harris_Lab_Spine_Identification.ipynb'),
 PosixPath('parameters_config_harris.py'),
 PosixPath('part_tri.txt'),
 PosixPath('part_vert.txt'),
 PosixPath('temp'),
 PosixPath('__pycache__')]

In [5]:
file_base = "full"
verts_file = f"{file_base}_vert.txt"
tri_file = f"{file_base}_tri.txt"
tri_array = nu.loadtxt(str(tri_file),dtype = "int",delimiter = " ")
verts_array =  nu.loadtxt(str(verts_file),dtype = "float",delimiter = " ")

In [6]:
mesh = tu.mesh_from_vertices_faces(vertices = verts_array, faces = tri_array)
mesh

<trimesh.Trimesh(vertices.shape=(226149, 3), faces.shape=(452370, 3))>

In [7]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.plot_objects(
    mesh,
    buffer = 0
)

/usr/local/lib/python3.8/dist-packages/traittypes/traittypes.py:97: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  warnings.warn(


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

# Decimating

In [8]:
mesh_dec = tu.decimate(mesh,decimation_ratio=0.25,)
mesh_dec = tu.largest_conn_comp(mesh_dec)
ipvu.plot_objects(
    mesh_dec,
    buffer=0
)

xvfb-run -n 7506 -s "-screen 0 800x600x24" meshlabserver $@  -i /neurd_packages/NEURD/Applications/Tutorials/Spine_Detection_On_Mesh_Branch/temp/neuron_11317.off -o /neurd_packages/NEURD/Applications/Tutorials/Spine_Detection_On_Mesh_Branch/temp/neuron_11317_decimated.off -s /neurd_packages/NEURD/Applications/Tutorials/Spine_Detection_On_Mesh_Branch/temp/decimation_meshlab_25133283.mls


Container(figure=Figure(box_center=[0.5, 0.5, 0.5], box_size=[1.0, 1.0, 1.0], camera=PerspectiveCamera(fov=45.…

# Setting Parameters for Spine Extraction

In [9]:
param_file = "./parameters_config_harris.py"

In [ ]:
vdi.set_parameters_obj_from_filepath(param_file)

In [ ]:
# checking to see the parameters were set
from neurd import spine_utils as spu
spu.shaft_mesh_n_faces_min_global

# Doing the spine extraction

In [ ]:
from neurd import spine_utils as spu

In [ ]:
spine_objs = spu.spine_objs_bare_minimum_filt_with_attr_from_branch_obj(
    mesh = mesh_dec,
    plot_filtered_spines = True,
)

# Inspecting the Spines

In [ ]:
idx = 10

sp_obj = spine_objs[idx]
ipvu.plot_objects(sp_obj.mesh)

In [ ]:
sp_obj.export()

# Exporting the figure

In [ ]:
spu.plot_spines_objs_with_head_neck_and_coordinates(
    spine_objs,
    mesh = mesh_dec,
)

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu
ipvu.save_to_html(f"{file_base}_spine_detection.html")

# Skeleton Generation

In [ ]:
from mesh_tools import skeleton_utils as sk

In [ ]:
from datasci_tools import ipyvolume_utils as ipvu

In [ ]:
skeleton_surface = sk.surface_skeleton(
    mesh_dec,
    plot = True,
    #n_surface_downsampling=3,
)

In [ ]:
skeleton = sk.skeleton_cgal(
    mesh_dec,
    plot = True,
    cgal_original_parameters = False,
    max_triangle_angle =1.91986,
    quality_speed_tradeoff=0.1,
    medially_centered_speed_tradeoff=0.2,#0.2,
    area_variation_factor=0.0001,
    max_iterations=500,#500,
    is_medially_centered=True,
    min_edge_length = 1,
    edge_length_multiplier = 0.002,
)

skeleton_no_cycles = sk.remove_cycles_from_skeleton(skeleton,verbose=True,)

sk_cgal,sk_removed = sk.clean_skeleton(
    skeleton_no_cycles,
    min_distance_to_junction = 3,
    return_removed_skeletons = True,
)

ipvu.plot_objects(
    mesh_dec,
    skeletons=[sk_cgal,sk.stack_skeletons(sk_removed)],
    skeletons_colors=["blue","red"],
    buffer=0
)